In [168]:
from pathlib import Path
import pandas as pd
csv_files = list((Path().glob('*.csv')))
dfs = []
for file in csv_files:
    df = pd.read_csv(file)
    split = str(file).split('_')
    architecture = split[0]
    tokenizer = split[-2]
    random_weights = False
    if('RANDOM' in split):
        random_weights = True
        
    df.insert(0, 'random_weights',random_weights)
    df.insert(0, 'tokenizer',tokenizer)
    df.insert(0, 'architecture',architecture)
    
    df['file_name'] = file
    dfs.append(df)
    # print(df.head())

main_df = pd.concat(dfs, ignore_index=True)

In [111]:
dsets = list(main_df.dataset.unique())

In [222]:
# main_df.head()

In [214]:
dfs = []
for dataset_index in range(6):
    df = main_df[
        # (main_df['random_weights']==True) & 
        # (main_df['architecture']=='DEBERTA') & 
        (main_df['dataset']==dsets[dataset_index])
    ][['architecture','dataset','tokenizer','random_weights','test_acc', 'test_f1', 'test_auroc_weighted','test_pr_auc']]\
    .sort_values('tokenizer')\
    .style\
    .hide(axis='index')\
    .format({"dataset": lambda x:x.lower()})\
    .background_gradient(subset=['test_acc', 'test_f1', 'test_auroc_weighted','test_pr_auc'], cmap='Blues')
    # .highlight_max(subset=['test_auroc_weighted'],color='lightgreen')\
    # .highlight_min(subset=['test_auroc_weighted'],color='#cd4f39')\
    dfs.append(df)
    # main_df.iloc[main_df.groupby(['dataset','tokenizer'])['test_auroc_weighted'].idxmax()]
    # main_df.iloc[main_df.groupby('dataset')['test_acc'].idxmax()]
    





In [215]:
dfs[0]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
DEBERTA,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.901200,0.901492,0.963793,0.961481
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.899560,0.901425,0.963367,0.961143
DNABERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,False,0.924600,0.925936,0.978707,0.978504
BERT,demo_coding_vs_intergenomic_seqs,DNABERTtokenizer,True,0.900920,0.900606,0.963096,0.960968
BERT,demo_coding_vs_intergenomic_seqs,Kmer7tokenizer,True,0.898840,0.901584,0.963771,0.962069
DEBERTA,demo_coding_vs_intergenomic_seqs,Kmer7tokenizer,True,0.901000,0.899452,0.963838,0.962015
DEBERTA,demo_coding_vs_intergenomic_seqs,Kmer8tokenizer,True,0.895320,0.895215,0.960163,0.959543
DEBERTA,demo_coding_vs_intergenomic_seqs,Kmer8tokenizer,False,0.911480,0.908926,0.971529,0.972589
BERT,demo_coding_vs_intergenomic_seqs,Kmer8tokenizer,True,0.898880,0.899595,0.963709,0.963179
DEBERTA,demo_coding_vs_intergenomic_seqs,Sentencepiece10tokenizer,False,0.910120,0.909076,0.969634,0.970662


In [216]:
dfs[1]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
DEBERTA,demo_human_or_worm,DNABERTtokenizer,True,0.954600,0.954912,0.991370,0.991745
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.954080,0.953925,0.990508,0.991099
DNABERT,demo_human_or_worm,DNABERTtokenizer,False,0.965800,0.965733,0.995246,0.995429
BERT,demo_human_or_worm,DNABERTtokenizer,True,0.953480,0.952710,0.991099,0.991605
BERT,demo_human_or_worm,Kmer7tokenizer,True,0.955720,0.955597,0.991495,0.991811
DEBERTA,demo_human_or_worm,Kmer7tokenizer,True,0.955400,0.955622,0.991569,0.991967
DEBERTA,demo_human_or_worm,Kmer8tokenizer,True,0.948320,0.948567,0.989329,0.989908
DEBERTA,demo_human_or_worm,Kmer8tokenizer,False,0.958080,0.958557,0.993469,0.993667
BERT,demo_human_or_worm,Kmer8tokenizer,True,0.939560,0.937430,0.988838,0.989494
DEBERTA,demo_human_or_worm,Sentencepiece10tokenizer,False,0.951520,0.951782,0.991599,0.991899


In [217]:
dfs[2]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
DEBERTA,human_enhancers_cohn,DNABERTtokenizer,True,0.730282,0.729191,0.804544,0.799972
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.728411,0.709558,0.806360,0.800327
DNABERT,human_enhancers_cohn,DNABERTtokenizer,False,0.741940,0.749476,0.828015,0.818516
BERT,human_enhancers_cohn,DNABERTtokenizer,True,0.726540,0.733819,0.800259,0.789622
BERT,human_enhancers_cohn,Kmer7tokenizer,True,0.727979,0.728370,0.803234,0.793617
DEBERTA,human_enhancers_cohn,Kmer7tokenizer,True,0.720927,0.724456,0.797471,0.788885
DEBERTA,human_enhancers_cohn,Kmer8tokenizer,True,0.705959,0.673381,0.791136,0.783248
DEBERTA,human_enhancers_cohn,Kmer8tokenizer,False,0.722798,0.750065,0.811191,0.796354
BERT,human_enhancers_cohn,Kmer8tokenizer,True,0.688256,0.649855,0.777960,0.769889
DEBERTA,human_enhancers_cohn,Sentencepiece10tokenizer,False,0.732585,0.721773,0.816845,0.805225


In [218]:
dfs[3]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
DEBERTA,human_enhancers_ensembl,DNABERTtokenizer,True,0.812851,0.810848,0.893935,0.895679
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.805392,0.798906,0.882320,0.884570
DNABERT,human_enhancers_ensembl,DNABERTtokenizer,False,0.866096,0.865867,0.941497,0.942554
BERT,human_enhancers_ensembl,DNABERTtokenizer,True,0.799871,0.811748,0.889409,0.892675
BERT,human_enhancers_ensembl,Kmer7tokenizer,True,0.796384,0.787175,0.872875,0.870909
DEBERTA,human_enhancers_ensembl,Kmer7tokenizer,True,0.795221,0.790319,0.870675,0.870248
DEBERTA,human_enhancers_ensembl,Kmer8tokenizer,True,0.793736,0.797026,0.866356,0.861359
DEBERTA,human_enhancers_ensembl,Kmer8tokenizer,False,0.834420,0.842516,0.918401,0.915678
BERT,human_enhancers_ensembl,Kmer8tokenizer,True,0.794575,0.789044,0.867052,0.863083
DEBERTA,human_enhancers_ensembl,Sentencepiece10tokenizer,False,0.852050,0.852165,0.927938,0.925206


In [219]:
dfs[4]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
DEBERTA,human_nontata_promoters,DNABERTtokenizer,True,0.835510,0.824225,0.924087,0.950049
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.831636,0.819552,0.921892,0.948723
DNABERT,human_nontata_promoters,DNABERTtokenizer,False,0.828979,0.815125,0.926020,0.951453
BERT,human_nontata_promoters,DNABERTtokenizer,True,0.823998,0.809398,0.920357,0.947520
BERT,human_nontata_promoters,Kmer7tokenizer,True,0.858424,0.851916,0.944867,0.963659
DEBERTA,human_nontata_promoters,Kmer7tokenizer,True,0.871043,0.867146,0.946988,0.965614
DEBERTA,human_nontata_promoters,Kmer8tokenizer,True,0.892517,0.894353,0.958166,0.970961
DEBERTA,human_nontata_promoters,Kmer8tokenizer,False,0.912553,0.913755,0.966116,0.977803
BERT,human_nontata_promoters,Kmer8tokenizer,True,0.814147,0.800333,0.924981,0.945811
DEBERTA,human_nontata_promoters,Sentencepiece10tokenizer,False,0.866726,0.862431,0.934840,0.958620


In [221]:
dfs[5]

architecture,dataset,tokenizer,random_weights,test_acc,test_f1,test_auroc_weighted,test_pr_auc
DEBERTA,human_ocr_ensembl,DNABERTtokenizer,True,0.718156,0.729480,0.794750,0.782172
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.713836,0.727302,0.789594,0.774474
DNABERT,human_ocr_ensembl,DNABERTtokenizer,False,0.768511,0.762344,0.850755,0.839602
BERT,human_ocr_ensembl,DNABERTtokenizer,True,0.714952,0.721322,0.788795,0.775356
BERT,human_ocr_ensembl,Kmer7tokenizer,True,0.726797,0.740552,0.800538,0.784340
DEBERTA,human_ocr_ensembl,Kmer7tokenizer,True,0.723764,0.725624,0.799970,0.786248
DEBERTA,human_ocr_ensembl,Kmer8tokenizer,True,0.719072,0.728043,0.791334,0.777627
DEBERTA,human_ocr_ensembl,Kmer8tokenizer,False,0.755579,0.779882,0.845863,0.833512
BERT,human_ocr_ensembl,Kmer8tokenizer,True,0.724165,0.734313,0.796511,0.782343
DEBERTA,human_ocr_ensembl,Sentencepiece10tokenizer,False,0.738756,0.751395,0.817400,0.801343
